In [30]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
import os

In [3]:
path_data = "/workspaces/ml_challenge/data"

In [4]:
filename_test = "test.csv"
filename_train = "train.csv"

In [5]:
def combine_path(path, filename):
    return os.path.join(path, filename)

In [6]:
def get_dataframe_by_path(filepath):
    return pd.read_csv(filepath)

In [7]:
filepath_train = combine_path(path_data, filename_train)
filepath_test = combine_path(path_data, filename_test)

In [8]:
df_train = get_dataframe_by_path(filepath_train)
df_test = get_dataframe_by_path(filepath_test)

In [9]:
df_test.info(), df_train.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

(None, (1460, 81))

In [10]:
def get_features(df):
    features = []
    for k,v in df.dtypes.items():
        if v == "int64":
            features.append(k)
    return features

In [11]:
features = get_features(df_test)
feature_to_predict = "SalePrice"

In [12]:
df_train = pd.concat((df_train[features], df_train[feature_to_predict]), axis=1)
df_test = df_test[features]

In [13]:
df_train.shape

(1460, 27)

In [14]:
df_train.describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,10516.828082,6.099315,5.575342,1971.267808,1984.865753,1162.626712,346.992466,5.844521,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,9981.264932,1.382997,1.112799,30.202904,20.645407,386.587738,436.528436,48.623081,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,334.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,882.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,1087.000000,0.000000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,1391.250000,728.000000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,4692.000000,2065.000000,572.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [15]:
df_train.sample(n=5)

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
1327,1328,20,6600,5,9,1982,2008,816,0,0,...,0,0,0,0,0,0,0,10,2008,130500
602,603,60,10041,8,5,1992,1992,927,988,0,...,120,150,0,0,0,0,0,2,2006,220000
581,582,20,12704,8,5,2008,2009,2042,0,0,...,0,90,0,0,0,0,0,8,2009,253293
843,844,90,8000,5,4,1961,1961,1800,0,0,...,0,0,0,0,0,0,0,7,2007,141000
1256,1257,20,14303,8,5,1994,1994,1987,0,0,...,262,36,0,0,0,0,0,8,2008,301500


In [16]:
df_train.value_counts()

Id    MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  SalePrice
1460  20          9937     5            6            1965       1965          1256      0         0             1256       1         1         3             1             6             0           736         68           0              0          0            0         0        6       2008    147500       1
1     60          8450     7            5            2003       2003          856       854       0             1710       2         1         3             1             8             0           0           61           0              0          0            0         0        2       2008    208500       1
2     20          9600     6            8            1976       1976    

In [17]:
list(df_train.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64')]

In [18]:
X = df_train.loc[:, df_train.columns != 'SalePrice']
Y = df_train.loc[:, 'SalePrice']

In [20]:
seed = 42
test_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [29]:
# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [1000, 2000, 4000],
    'max_depth': [6, 8, 10, 12],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8]
}

In [25]:
model = xgb.XGBRegressor(random_state=42)

In [32]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
best_model = xgb.XGBRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.6; total time=   2.5s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.6; total time=   2.5s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.6; total time=   2.5s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.7; total time=   2.6s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.8; total time=   2.5s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.7; total time=   2.6s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.7; total time=   2.7s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, n_estimators=1000, subsample=0.8; total time=   2.8s
[CV] END colsample_bytree=0.6, learning_rate=0.01, max_depth=6, 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=4000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [26]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=4000,
             n_jobs=None, num_parallel_tree=None, ...)

In [33]:
y_pred = best_model.predict(X_test)

In [34]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Mean Absolute Error (MAE): 17626.859877996576
Mean Squared Error (MSE): 748362355.5780565
Root Mean Squared Error (RMSE): 27356.212376315118
R-squared (R²): 0.8931721448898315
Mean Absolute Percentage Error (MAPE): 10.39378033815989%


In [35]:
path_models = os.path.join(path_data, "models/")

In [36]:
str_date = datetime.today().strftime('%Y_%m_%d_%H_%M_%S')

In [37]:
filename_models = f"model_{str_date}.json"
filepath_models = combine_path(path_models, filename_models)

In [38]:
best_model.save_model(filepath_models)

In [ ]:
dif = y_pred - y_test

In [ ]:
dif.min(), dif.mean(), dif.max()

In [ ]:
df_train.columns, df_test.columns


In [40]:
def create_submission(model, df, str_date):
    path_submission = os.path.join(path_data, "submission/")
    filename = f"submission_{str_date}.csv"
    filepath_submission = combine_path(path_submission, filename)
    ids_series = df["Id"]
    predicted_prices = model.predict(df)
    predicted_prices_series = pd.Series(predicted_prices, name="SalePrice")

    submission = pd.DataFrame({
        'id': ids_series,
        'SalePrice': predicted_prices_series
    }).set_index('id')
    submission.to_csv(path_or_buf=filepath_submission)

In [41]:
create_submission(best_model, df_test, str_date)

In [ ]:
submission

In [42]:
best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=4000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [44]:
best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=4000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)